In [244]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [245]:
df = pd.read_csv('S&PDataset_final.csv')
df.shape

(11553, 26)

In [246]:
row_count_per_gvkey = df['GVKEY'].value_counts()
row_count_per_gvkey

GVKEY
14282    26
13824    26
11506    26
11465    26
11456    26
         ..
36191     4
37460     4
40614     2
43197     1
41818     1
Name: count, Length: 497, dtype: int64

In [303]:
companies_to_drop = row_count_per_gvkey[row_count_per_gvkey < 25].index
companies_to_drop

Index([  3851,  62689,   5568,  28877, 126136, 137131, 126554, 141459, 139665,
       138743,
       ...
        34636,  34443,  35077,  35168,  36190,  36191,  37460,  40614,  43197,
        41818],
      dtype='int64', name='GVKEY', length=137)

In [304]:
df = df[~df['GVKEY'].isin(companies_to_drop)]

In [305]:
df['conm'].nunique()

360

In [250]:
df.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco          418
act         1814
aqc          938
at             0
capx         153
ceq            0
dt          1445
ebit           0
icapt          1
lct         1801
ni             0
revt           0
seq            0
xrd         3946
xrdp        3966
costat         0
sic            0
dtype: int64

In [251]:
# Counting total financial companies

rows_with_sic_starting_6 = df[df['sic'].astype(str).str.startswith('6')]
grouped_by_gvkey = rows_with_sic_starting_6.groupby('GVKEY')
unique_gvkeys = grouped_by_gvkey.groups.keys()
len(unique_gvkeys)


69

In [252]:
# # Counting financial companies that have incomplete data

# # filtered_df = df[~df['fyear'].between(1999, 2022)]
# filtered_df_starting_6 = df[df['sic'].astype(str).str.startswith('6')]
# grouped_by_gvkey = filtered_df_starting_6.groupby('GVKEY')
# unique_gvkeys_starting_6 = grouped_by_gvkey.groups.keys()
# len(unique_gvkeys_starting_6)

69

In [307]:
# Dropping all companies that have incomplete data and financial companies

# df_within_year_range = df[df['fyear'].between(1999, 2022)]
df_no_sic_starting_6 = df[~df['sic'].astype(str).str.startswith('6')]

In [306]:
df_no_sic_starting_6['GVKEY'].nunique()

291

In [255]:
df_no_sic_starting_6['GVKEY'].value_counts()

GVKEY
8543      26
63172     26
10247     26
10407     26
10453     26
          ..
15855     25
16721     25
25313     25
27965     25
126721    25
Name: count, Length: 291, dtype: int64

In [308]:
print("Shape: ", df_no_sic_starting_6.shape)
print("Total remaining companies: ", df_no_sic_starting_6['conm'].nunique())

Shape:  (7523, 26)
Total remaining companies:  291


In [309]:
df_no_sic_starting_6.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco            2
act          295
aqc          751
at             0
capx          11
ceq            0
dt           987
ebit           0
icapt          1
lct          282
ni             0
revt           0
seq            0
xrd         2701
xrdp        2720
costat         0
sic            0
dtype: int64

In [258]:
df_no_sic_starting_6['GVKEY'].value_counts()

GVKEY
8543      26
63172     26
10247     26
10407     26
10453     26
          ..
15855     25
16721     25
25313     25
27965     25
126721    25
Name: count, Length: 291, dtype: int64

In [310]:
# Counting the unique companies that have missing values for total current assets and liabilities

grouped_by_gvkey = df_no_sic_starting_6.groupby('GVKEY')

missing_act = grouped_by_gvkey['act'].apply(lambda x: x.isnull().sum())
missing_lct = grouped_by_gvkey['lct'].apply(lambda x: x.isnull().sum())

missing_companies = []

for gvkey, count in missing_act.items():
    if count != 0 and count > 2:
        missing_companies.append(gvkey)
    elif count==1:
        lct_act = gvkey

print(("{total} companies with missing act and lct values: {list}").format(total = len(missing_companies), list = missing_companies))


14 companies with missing act and lct values: [2176, 3835, 4839, 5047, 5073, 6669, 8253, 8823, 10519, 11456, 12459, 12635, 25340, 66065]


In [260]:
df_no_sic_starting_6[df_no_sic_starting_6['GVKEY'] == lct_act]['act']

4514        NaN
4515        NaN
4516    17238.0
4517    17275.0
4518    17441.0
4519    21382.0
4520    25901.0
4521    25781.0
4522    26152.0
4523    22890.0
4524    11076.0
4525     5773.0
4526     5981.0
4527     7131.0
4528     6315.0
4529     6590.0
4530     6878.0
4531     6086.0
4532     7260.0
4533     4344.0
4534     4299.0
4535     4824.0
4536     7117.0
4537     6083.0
4538     7220.0
4539     5585.0
Name: act, dtype: float64

In [261]:
missing_act

GVKEY
1045      0
1075      0
1078      0
1161      0
1209      0
         ..
119314    0
121077    0
121718    0
125595    0
126721    0
Name: act, Length: 291, dtype: int64

In [225]:
missing_companies

[2176,
 3835,
 4839,
 5047,
 5073,
 6669,
 8253,
 8823,
 10519,
 11456,
 12459,
 12635,
 25340,
 66065]

In [226]:
lct_act

8543

In [227]:
df_no_sic_starting_6['GVKEY'].nunique()

291

In [262]:
# Dropping companies with no act or lct values
df_final = df_no_sic_starting_6[~df_no_sic_starting_6['GVKEY'].isin(missing_companies)]
df_final['GVKEY'].nunique()

277

In [263]:
# Substituing null values for aqcuisitions to 0

df_final['aqc'].fillna(0, inplace=True)

In [264]:
# Dropping 'total debt'

df_final = df_final.drop(columns=['dt'])
df_final.columns

Index(['GVKEY', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'conm', 'curcd', 'aco', 'act', 'aqc', 'at', 'capx', 'ceq', 'ebit',
       'icapt', 'lct', 'ni', 'revt', 'seq', 'xrd', 'xrdp', 'costat', 'sic'],
      dtype='object')

In [265]:
df_final.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco            2
act            2
aqc            0
at             0
capx          30
ceq            0
ebit           0
icapt          1
lct            2
ni             0
revt           0
seq            0
xrd         2532
xrdp        2551
costat         0
sic            0
dtype: int64

In [268]:
df_final[df_final['fyear'] == 2023]

,GVKEY,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,aco,...,ebit,icapt,lct,ni,revt,seq,xrd,xrdp,costat,sic
24,1045,31/12/2023,2023,INDL,C,D,STD,AMERICAN AIRLINES GROUP INC,USD,658.000,...,4013.000,30520.000,22062.000,822.000,52788.000,-5202.000,NaN,NaN,A,4512
75,1078,31/12/2023,2023,INDL,C,D,STD,ABBOTT LABORATORIES,USD,2256.000,...,6793.000,53375.000,13841.000,5723.000,40109.000,38603.000,2741.000,2888.000,A,3845
101,1161,31/12/2023,2023,INDL,C,D,STD,ADVANCED MICRO DEVICES,USD,1259.000,...,629.000,58144.000,6689.000,854.000,22680.000,55892.000,5872.000,5005.000,A,3674
127,1209,30/09/2023,2023,INDL,C,D,STD,AIR PRODUCTS & CHEMICALS INC,USD,436.800,...,2704.400,25722.700,3895.800,2300.200,12600.000,14312.900,105.600,102.900,A,2810
153,1300,31/12/2023,2023,INDL,C,D,STD,HONEYWELL INTERNATIONAL INC,USD,1462.000,...,7084.000,33900.000,18539.000,5658.000,36662.000,15856.000,1456.000,1478.000,A,9997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9686,119314,31/12/2023,2023,INDL,C,D,STD,BOOKING HOLDINGS INC,USD,1070.000,...,6793.000,10112.000,13330.000,4289.000,21365.000,-2744.000,NaN,NaN,A,4700
9711,121077,30/09/2023,2023,INDL,C,D,STD,F5 INC,USD,180.631,...,559.618,3039.797,1472.781,394.948,2813.169,2800.232,540.285,543.368,A,7373
9761,121718,31/12/2023,2023,INDL,C,D,STD,JUNIPER NETWORKS INC,USD,572.600,...,568.800,6192.400,2104.000,310.200,5564.500,4492.700,1144.400,1036.100,A,3576
9811,125595,31/12/2023,2023,INDL,C,D,STD,AKAMAI TECHNOLOGIES INC,USD,216.114,...,706.497,8910.190,836.038,547.629,3811.920,4597.155,406.048,391.434,A,7370


In [266]:
df_final = df_final[df_final['fyear'] != 1998]

In [269]:
df_final = df_final[df_final['fyear'] != 2023]

In [272]:
df_final = df_final[df_final['GVKEY'] != 1045]

In [273]:
df_final['GVKEY'].value_counts()

GVKEY
1075      24
12540     24
14225     24
13824     24
13714     24
          ..
6730      24
6733      24
6774      24
6829      24
126721    24
Name: count, Length: 276, dtype: int64

In [153]:
# df_final.to_csv('final_dataset.csv')

In [275]:
df_final.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco            0
act            1
aqc            0
at             0
capx          22
ceq            0
ebit           0
icapt          0
lct            1
ni             0
revt           0
seq            0
xrd         2322
xrdp        2339
costat         0
sic            0
dtype: int64

# Imputation for ACT LCT values

In [276]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Step 1: Filter rows where gvkey is equal to lct_act
df_subset = df_final[df_final['GVKEY'] == lct_act].copy()
df_subset = df_final.drop(columns=['datadate','indfmt','consol','popsrc','datafmt','conm','curcd','costat', 'xrd', 'xrdp', 'capx'])


# Step 2: Split the dataset into two parts
df_missing = df_subset[df_subset[['act', 'lct']].isnull().any(axis=1)]  # Rows with missing values
df_complete = df_subset.dropna(subset=['act', 'lct'])  # Rows with complete data

# Step 3: Train a linear regression model for 'act'
X_act = df_complete.drop(['act', 'lct'], axis=1)  # Features for 'act' (excluding 'lct' and 'act')
y_act = df_complete['act']  # Target variable for 'act'

model_act = LinearRegression()
model_act.fit(X_act, y_act)

# Step 4: Use the trained model to predict missing values for 'act'
X_missing_act = df_missing.drop(['act', 'lct'], axis=1)  # Features from rows with missing values for 'act'
imputed_values_act = model_act.predict(X_missing_act)

# Step 5: Replace missing values in 'act' with predicted values
df_missing['act'] = imputed_values_act

# Step 6: Train a linear regression model for 'lct'
X_lct = df_complete.drop(['act', 'lct'], axis=1)  # Features for 'lct' (excluding 'lct' and 'act')
y_lct = df_complete['lct']  # Target variable for 'lct'

model_lct = LinearRegression()
model_lct.fit(X_lct, y_lct)

# Step 7: Use the trained model to predict missing values for 'lct'
X_missing_lct = df_missing.drop(['act', 'lct'], axis=1)  # Features from rows with missing values for 'lct'
imputed_values_lct = model_lct.predict(X_missing_lct)

# Step 8: Replace missing values in 'lct' with predicted values
df_missing['lct'] = imputed_values_lct

# Update the original DataFrame with imputed values
df_final.update(df_missing)

In [277]:
df_final[df_final['GVKEY'] == lct_act]

,GVKEY,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,aco,...,ebit,icapt,lct,ni,revt,seq,xrd,xrdp,costat,sic
4515,8543,31/12/1999,1999,INDL,C,D,STD,ALTRIA GROUP INC,USD,2454.0,...,13781.0,27531.0,19548.490503,7675.0,61751.0,15305.0,522.0,506.0,A,2111
4516,8543,31/12/2000,2000,INDL,C,D,STD,ALTRIA GROUP INC,USD,2517.0,...,14567.0,34159.0,26976.000000,8510.0,63276.0,15005.0,538.0,522.0,A,2111
4517,8543,31/12/2001,2001,INDL,C,D,STD,ALTRIA GROUP INC,USD,2751.0,...,16295.0,42284.0,20653.000000,8560.0,72944.0,19620.0,647.0,538.0,A,2111
4518,8543,31/12/2002,2002,INDL,C,D,STD,ALTRIA GROUP INC,USD,2610.0,...,17145.0,45199.0,19082.000000,11102.0,62182.0,19478.0,686.0,647.0,A,2111
4519,8543,31/12/2003,2003,INDL,C,D,STD,ALTRIA GROUP INC,USD,2809.0,...,16223.0,51000.0,21393.000000,9204.0,60704.0,25077.0,762.0,686.0,A,2111
4520,8543,31/12/2004,2004,INDL,C,D,STD,ALTRIA GROUP INC,USD,4362.0,...,16182.0,54161.0,23574.000000,9416.0,63963.0,30714.0,809.0,756.0,A,2111
4521,8543,31/12/2005,2005,INDL,C,D,STD,ALTRIA GROUP INC,USD,3578.0,...,17185.0,57716.0,26158.000000,10435.0,68920.0,35707.0,943.0,809.0,A,2111
4522,8543,31/12/2006,2006,INDL,C,D,STD,ALTRIA GROUP INC,USD,2876.0,...,17893.0,57025.0,26047.000000,12022.0,70324.0,39619.0,1005.0,943.0,A,2111
4523,8543,31/12/2007,2007,INDL,C,D,STD,ALTRIA GROUP INC,USD,2498.0,...,13912.0,26517.0,18782.000000,9786.0,38051.0,18554.0,631.0,586.0,A,2111
4524,8543,31/12/2008,2008,INDL,C,D,STD,ALTRIA GROUP INC,USD,2047.0,...,5014.0,9667.0,7642.000000,4930.0,15957.0,2828.0,232.0,269.0,A,2111


In [293]:
missing_capx = df_final[df_final['capx'].isnull()].groupby('GVKEY').size()
missing_capx

GVKEY
1078     3
1878     3
3505     3
8247     3
8549     2
14934    1
25313    3
61574    2
66446    2
dtype: int64

# Imputation of capx column

In [294]:
from sklearn.impute import KNNImputer

# Assuming your data is stored in a DataFrame called df

# Select columns with missing values
cols_with_missing = ['capx']

# Initialize KNN imputer with k=5
imputer = KNNImputer(n_neighbors=5)

# Group the DataFrame by 'GVKEY'
grouped = df_final.groupby('GVKEY')

# Impute missing values for each group separately
for name, group in grouped:
    # Impute missing values for the current group
    group[cols_with_missing] = imputer.fit_transform(group[cols_with_missing])
    # Update the imputed values in the main DataFrame
    df_final.loc[group.index, cols_with_missing] = group[cols_with_missing]

# Print the DataFrame to verify the imputed values
print(df_final)

       GVKEY    datadate  fyear indfmt consol popsrc datafmt  \
26      1075  31/12/1999   1999   INDL      C      D     STD   
27      1075  31/12/2000   2000   INDL      C      D     STD   
28      1075  31/12/2001   2001   INDL      C      D     STD   
29      1075  31/12/2002   2002   INDL      C      D     STD   
30      1075  31/12/2003   2003   INDL      C      D     STD   
...      ...         ...    ...    ...    ...    ...     ...   
9901  126721  31/12/2018   2018   INDL      C      D     STD   
9902  126721  31/12/2019   2019   INDL      C      D     STD   
9903  126721  31/12/2020   2020   INDL      C      D     STD   
9904  126721  31/12/2021   2021   INDL      C      D     STD   
9905  126721  31/12/2022   2022   INDL      C      D     STD   

                            conm curcd      aco  ...     ebit      icapt  \
26    PINNACLE WEST CAPITAL CORP   USD   68.725  ...  578.777   4411.785   
27    PINNACLE WEST CAPITAL CORP   USD  103.791  ...  675.971   4337.797   
28 

In [295]:
df_final.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco            0
act            0
aqc            0
at             0
capx           0
ceq            0
ebit           0
icapt          0
lct            0
ni             0
revt           0
seq            0
xrd         2322
xrdp        2339
costat         0
sic            0
dtype: int64

In [296]:
df_final['GVKEY'].value_counts()

GVKEY
1075      24
12540     24
14225     24
13824     24
13714     24
          ..
6730      24
6733      24
6774      24
6829      24
126721    24
Name: count, Length: 276, dtype: int64

In [298]:
df_final.to_csv('final_imputed_dataset1.csv')

In [302]:
df_final["GVKEY"].nunique()

276